## Currency Converter

### Description
This is a simple currency converter that converts an amount from one currency to another using the latest exchange rates. The user can input the amount, the source currency, and the target currency, and the program will output the converted amount.

#### Flow of the program:
- Create a tool to fetch the latest currency exchange rates from an ExchangeRate-API.
- Create a function to convert the amount from the source currency to the target currency using the fetched exchange rates.
- 

In [85]:
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
import os

In [86]:
load_dotenv()
API_KEY = os.getenv("EXCHANGE_RATE_API_KEY")

In [87]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_rate(base_currency: str, target_currency: str) -> float:
    """
    To fetch the current conversion rate between two currencies use this tool.
    this tool will take two parameters base_currency and target_currency and return the conversion rate.
    """
    url = f"https://v6.exchangerate-api.com/v6/{API_KEY}/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    data = response.json()
    return data

# here we have used Annotated[float, InjectedToolArg] to inject the conversion rate into the tool
# we are saying llm that do not call this tool directly, but use the conversion rate from the previous tool and i as a (developer/runtime) will take care of calling this tool
@tool
def convert_currency(base_currency: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    To convert the currency from base currency to target currency using the current converstion rate use this tool.
    """
    return base_currency * conversion_rate

In [88]:
result = get_conversion_rate.invoke({
    "base_currency": "USD",
    "target_currency": "INR"
})

In [89]:
final_result = convert_currency.invoke({
    "base_currency": 100,
    "conversion_rate": result["conversion_rate"]
})

In [90]:
final_result

8543.01

In [91]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [92]:
llm_with_tools = llm.bind_tools([get_conversion_rate, convert_currency])

In [101]:
messages = [
    HumanMessage(
        content="What is the current conversion rate between INR and USD and convert 100 INR to USD?"
    )
]

In [102]:
response = llm_with_tools.invoke(messages)

In [103]:
messages.append(response)

In [104]:
response.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': '4e2d5799-4353-445e-b81f-631a54886ab2',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'base_currency': 100.0},
  'id': 'be38b2e4-6395-4f2a-bc38-dc28173ea967',
  'type': 'tool_call'}]

In [105]:
import json

for tool_call in response.tool_calls:
    # execute the 1st tool call and get the result of conversion rate
    # and then pass the conversion rate to the 2nd tool call 
    if tool_call['name'] == "get_conversion_rate":
        conversion_rate_tool_message = get_conversion_rate.invoke(tool_call)
        # fetch the conversion rate from the tool message
        conversion_rate = json.loads(conversion_rate_tool_message.content)["conversion_rate"]
        # append this tool message to the messages list
        messages.append(conversion_rate_tool_message)
    elif tool_call['name'] == "convert_currency":
        # fetch the current argument and add the conversion rate to the argument
        tool_call['args']['conversion_rate'] = conversion_rate
        convert_tool_message = convert_currency.invoke(tool_call)
        # append this tool message to the messages list
        messages.append(convert_tool_message)

In [106]:
messages

[HumanMessage(content='What is the current conversion rate between INR and USD and convert 100 INR to USD?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert_currency', 'arguments': '{"base_currency": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-dedac525-32ac-4824-a942-ea14c80c477f-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '4e2d5799-4353-445e-b81f-631a54886ab2', 'type': 'tool_call'}, {'name': 'convert_currency', 'args': {'base_currency': 100.0, 'conversion_rate': 0.01171}, 'id': 'be38b2e4-6395-4f2a-bc38-dc28173ea967', 'type': 'tool_call'}], usage_metadata={'input_tokens': 104, 'output_tokens': 19, 'total_tokens': 123, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchange

In [107]:
final_result = llm_with_tools.invoke(messages)

In [108]:
final_result

AIMessage(content='The current conversion rate between INR and USD is 0.01171. 100 INR is equal to 1.171 USD.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a64a50d3-0fe8-4e68-bfc2-182755b1047e-0', usage_metadata={'input_tokens': 264, 'output_tokens': 35, 'total_tokens': 299, 'input_token_details': {'cache_read': 0}})